In [2]:
import sys
sys.path.append('../../..')
from Dataset import interpolation, get_data_list
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.transforms import InterpolationMode
import numpy as np
import torch
from PIL import Image
import pyopenpose as op
import cv2

In [3]:
class PersonInWifiDataset(Dataset):
    def __init__(self, 
                 data_root: str, split: str = 'train', 
                 crop_size: tuple = (192, 256), interpolation: float = -1,
                 amp_offset: float = 60000, pha_offset: float = 28000,
                 skeleton_model: str = '/root/openpose/models'):
        super().__init__()
        self.data_root = data_root
        self.crop_size = crop_size
        self.data_list = get_data_list(split, data_root)
        self.split = split
        self.data_struct = {}
        for data in self.data_list:
            env = data[0].split('/')[0]
            if env not in self.data_struct:
                self.data_struct[env] = []
            self.data_struct[env].append(data)

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(
                crop_size, interpolation=InterpolationMode.NEAREST),
        ])
        self.interpolation = interpolation
        self.amp_offset = amp_offset
        self.pha_offset = pha_offset

        # Skeleton
        params = {
            "model_folder": skeleton_model,
            "heatmaps_add_parts": True,
            "heatmaps_add_PAFs": True,
            "heatmaps_add_bkg": True,
            "heatmaps_scale": 2,
        }
        self.opWrapper = op.WrapperPython()
        self.opWrapper.configure(params)
        self.opWrapper.start()

    def normalize(self, x: torch.Tensor, mean: float = 0, std: float = 0.5):
        return ((x - x.mean()) / x.std()) * std + mean

    def get_data(self, mask_path, data_path)->tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        # Mask
        mask = Image.open(f'{self.data_root}/{mask_path}_mask.png').convert('L')
        mask = self.transform(mask).float()
        # Skeleton
        img = cv2.imread(f'{self.data_root}/{mask_path}.jpg')
        img = cv2.resize(img, self.crop_size[::-1])
        datum = op.Datum()
        datum.cvInputData = img
        self.opWrapper.emplaceAndPop(op.VectorDatum([datum]))
        heatmaps = datum.poseHeatMaps.copy()
        heatmaps = (heatmaps).astype(dtype='uint8')
        jhm = heatmaps[:25]
        paf = heatmaps[26:]

        print(jhm)
        print("###")
        print(paf)

        # CSI
        data = np.load(f'{self.data_root}/{data_path}.npz')
        amp = torch.from_numpy(data['mag'].astype(np.float32)/self.amp_offset)
        pha = torch.from_numpy(data['pha'].astype(np.float32)/self.pha_offset)
        amp, pha = self.normalize(amp), self.normalize(pha)
        if self.split == 'train' and np.random.random() < self.interpolation:
            amp, pha = interpolation(amp, pha)
        return amp, pha, mask

    def __getitem__(self, idx):
        data_path = self.data_list[idx]
        print(data_path)
        env = data_path.split('/')[0]
        print(env)
        mask_path = data_path.replace('npy', 'img')
        amp, pha, mask = self.get_data(mask_path, data_path)    
        
        return amp, pha, mask

In [4]:
dataset = PersonInWifiDataset(data_root='/root/CSI_Dataset')

Starting OpenPose Python Wrapper...
Auto-detecting all available GPUs... Detected 2 GPU(s), using 2 of them starting at GPU 0.


In [7]:
dataset[1]

Env0/npy/N/1_posi/240507_143709/1715063841284541555
Env0
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
###
[[[129 129 129 ... 129 129 129]
  [129 129 129 ... 129 129 129]
  [129 129 129 ... 129 129 129]
  ...
  [129 129 129 ... 129 129 129]
  [129 129 129 ... 129 129 129]

(tensor([[[-0.5482, -0.5547, -0.4975,  ..., -0.9375, -0.9375, -0.9375],
          [-0.4289, -0.4454, -0.4228,  ..., -0.9375, -0.9375, -0.9375],
          [-0.4269, -0.4150, -0.3824,  ..., -0.9375, -0.9375, -0.9375],
          [-0.5575, -0.6108, -0.6053,  ..., -0.9375, -0.9375, -0.9375],
          [-0.2668, -0.2550, -0.2538,  ..., -0.9375, -0.9375, -0.9375],
          [-0.3196, -0.3567, -0.4089,  ..., -0.9375, -0.9375, -0.9375]],
 
         [[-0.5849, -0.5207, -0.5185,  ..., -0.9375, -0.9375, -0.9375],
          [-0.3933, -0.3723, -0.3556,  ..., -0.9375, -0.9375, -0.9375],
          [-0.3534, -0.3637, -0.3603,  ..., -0.9375, -0.9375, -0.9375],
          [-0.4899, -0.4847, -0.4692,  ..., -0.9375, -0.9375, -0.9375],
          [-0.3777, -0.4088, -0.3954,  ..., -0.9375, -0.9375, -0.9375],
          [-0.5341, -0.5735, -0.5637,  ..., -0.9375, -0.9375, -0.9375]],
 
         [[-0.5350, -0.4957, -0.4728,  ..., -0.9375, -0.9375, -0.9375],
          [-0.4402, -0.4361, -0.4168,  ..., -0.9375, -0.93